In [56]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import linear_model
import statsmodels.api as sm

In [82]:
df = pd.read_excel('Data/bereinigte_DATEN_ano.xlsx')

In [83]:
df.head()

,Unnamed: 0,V1,athlete,gender,run,location,date,TurnNr,MinimalRadius,VelocityAtMinRadius,...,azitonextnext,HORIZONTALGATEDISTANCE,VERTICALGATEDISTANCE,regularbs1orfs2,steepness_A,filter_$,goofy2_regular1,@2Ferse_3Zehe_4Ferse,Zehen1_Fersen2,DistanzSchätz
0,0,35,3,1,1,Zermatt,200915,2,10.63,53.6,...,341.45477,8.515579,20.414966,1,2,0,1,2,2,30.635388
1,1,36,3,1,2,Zermatt,200915,2,11.05,52.3,...,341.45477,8.515579,20.414966,1,2,0,1,2,2,30.635388
2,2,37,3,1,3,Zermatt,200915,2,10.99,52.8,...,341.45477,8.515579,20.414966,1,2,0,1,2,2,30.635388
3,3,38,3,1,4,Zermatt,200915,2,11.41,52.2,...,341.45477,8.515579,20.414966,1,2,0,1,2,2,30.635388
4,4,39,5,1,1,Zermatt,200915,2,11.31,49.7,...,341.45477,8.515579,20.414966,1,2,0,2,3,1,30.635388


In [84]:
df_clean = df.drop(["Unnamed: 0", "V1", "date", "vectortonext", "vector2Dtonext", "vectortonextnorm", "vector2Dtonextnorm", "projpttonext", "vectortonextnext", "vector2Dtonextnext", "vectortonextnextnorm", "vector2Dtonextnextnorm", 'DistanzSchätz','Zehen1_Fersen2', '@2Ferse_3Zehe_4Ferse', 'filter_$', 'goofy2_regular1', 'athlete', 'run', 'Unnamed0', 'location', 'regularbs1orfs2', 'steepness_A', 'azitonextnext', 'dts2Dtonextnext'], axis=1)

In [85]:
# remove all time parameters
df_clean = df_clean.drop(["TimeStarttoEnd", "TimeStartToMinRad", "TimeMinRadToEnd", "GlideTime_endtostart", "TimeStarttoEnd_2"], axis =1)

In [86]:
df_clean.columns

Index(['gender', 'TurnNr', 'MinimalRadius', 'VelocityAtMinRadius',
       'VelocityAtTurnEntry', 'VelocityAtTurnExit', 'Initiation', 'Completion',
       'Gliding', 'GATEDISTANCE', 'dist2Dtonext', 'STEEPNESS', 'azitonext',
       'slopechangetonext', 'angletonext', 'projtonext', 'offsettonext',
       'dist3Dtonextnext', 'slopetonextnext', 'HORIZONTALGATEDISTANCE',
       'VERTICALGATEDISTANCE'],
      dtype='object')

In [87]:
df.columns

Index(['Unnamed: 0', 'V1', 'athlete', 'gender', 'run', 'location', 'date',
       'TurnNr', 'MinimalRadius', 'VelocityAtMinRadius', 'VelocityAtTurnEntry',
       'VelocityAtTurnExit', 'TimeStarttoEnd', 'TimeStartToMinRad',
       'TimeMinRadToEnd', 'GlideTime_endtostart', 'TimeStarttoEnd_2',
       'Initiation', 'Completion', 'Gliding', 'Unnamed0', 'vectortonext',
       'vector2Dtonext', 'GATEDISTANCE', 'dist2Dtonext', 'vectortonextnorm',
       'vector2Dtonextnorm', 'STEEPNESS', 'azitonext', 'slopechangetonext',
       'angletonext', 'projtonext', 'projpttonext', 'offsettonext',
       'vectortonextnext', 'vector2Dtonextnext', 'dist3Dtonextnext',
       'dts2Dtonextnext', 'vectortonextnextnorm', 'vector2Dtonextnextnorm',
       'slopetonextnext', 'azitonextnext', 'HORIZONTALGATEDISTANCE',
       'VERTICALGATEDISTANCE', 'regularbs1orfs2', 'steepness_A', 'filter_$',
       'goofy2_regular1', '@2Ferse_3Zehe_4Ferse', 'Zehen1_Fersen2',
       'DistanzSchätz'],
      dtype='object')

In [96]:
x = df_clean.iloc[:,0:15]
y = df.iloc[:, 16:17]

In [97]:
len(x.columns)

15

In [98]:
x.head()

,gender,TurnNr,MinimalRadius,VelocityAtMinRadius,VelocityAtTurnEntry,VelocityAtTurnExit,Initiation,Completion,Gliding,GATEDISTANCE,dist2Dtonext,STEEPNESS,azitonext,slopechangetonext,angletonext
0,1,2,10.63,53.6,55.7,55.2,0.421053,0.421053,0.157895,22.119809,21.248354,-16.13636,317.453817,6.171654,44.982881
1,1,2,11.05,52.3,57.7,55.6,0.384615,0.461538,0.153846,22.119809,21.248354,-16.13636,317.453817,6.171654,44.982881
2,1,2,10.99,52.8,57.0,58.2,0.384615,0.461538,0.153846,22.119809,21.248354,-16.13636,317.453817,6.171654,44.982881
3,1,2,11.41,52.2,54.5,54.9,0.500000,0.363636,0.136364,22.119809,21.248354,-16.13636,317.453817,6.171654,44.982881
4,1,2,11.31,49.7,50.0,58.1,0.519231,0.346154,0.134615,22.119809,21.248354,-16.13636,317.453817,6.171654,44.982881


In [91]:
# normalize the values
normalized_df=(x-x.mean())/x.std()

In [92]:
normalized_df.head()

,gender,TurnNr,MinimalRadius,VelocityAtMinRadius,VelocityAtTurnEntry,VelocityAtTurnExit,Initiation,Completion,Gliding,GATEDISTANCE,...,STEEPNESS,azitonext,slopechangetonext,angletonext,projtonext,offsettonext,dist3Dtonextnext,slopetonextnext,HORIZONTALGATEDISTANCE,VERTICALGATEDISTANCE
0,-1.591172,-1.540995,-0.617841,0.724101,0.686453,0.633241,-0.232642,0.150793,0.103779,-0.660796,...,-0.40208,0.789873,1.353806,0.642505,-0.747904,0.563498,-0.809284,-0.969503,0.516865,-0.894234
1,-1.591172,-1.540995,-0.544422,0.496945,1.056166,0.708479,-0.656975,0.731953,0.059421,-0.660796,...,-0.40208,0.789873,1.353806,0.642505,-0.747904,0.563498,-0.809284,-0.969503,0.516865,-0.894234
2,-1.591172,-1.540995,-0.554910,0.584313,0.926766,1.197532,-0.656975,0.731953,0.059421,-0.660796,...,-0.40208,0.789873,1.353806,0.642505,-0.747904,0.563498,-0.809284,-0.969503,0.516865,-0.894234
3,-1.591172,-1.540995,-0.481490,0.479472,0.464625,0.576811,0.686747,-0.673399,-0.132124,-0.660796,...,-0.40208,0.789873,1.353806,0.642505,-0.747904,0.563498,-0.809284,-0.969503,0.516865,-0.894234
4,-1.591172,-1.540995,-0.498971,0.042634,-0.367229,1.178722,0.910701,-0.924355,-0.151278,-0.660796,...,-0.40208,0.789873,1.353806,0.642505,-0.747904,0.563498,-0.809284,-0.969503,0.516865,-0.894234


In [93]:
# with sklearn
regr = linear_model.LinearRegression()
regr.fit(x, y)

LinearRegression()

In [94]:
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

Intercept: 
 [1.96641317]
Coefficients: 
 [[ 6.42418017e-02 -6.75443114e-03  1.24083742e-02 -1.03073863e-02
  -1.19791699e-02 -2.82968059e-03 -1.78197213e-01 -5.18497044e-01
   6.96694257e-01  1.47319908e-01 -1.32696667e-01  1.23105853e-02
   6.86380402e-05  1.36302915e-02  8.67867579e-03  5.30388687e-02
   2.45008832e-02 -2.36302759e-03 -1.18529648e-02  3.26822720e-02
  -5.65253131e-02]]


In [95]:
# prediction with sklearn
New_VelocityAtTurnEntry = 2.75
New_MinimalRadius = 5.3
print ('Predicted Stock Index Price: \n', regr.predict([[New_VelocityAtTurnEntry ,New_MinimalRadius]]))

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 21 is different from 2)

In [ ]:
# with statsmodels
x = sm.add_constant(x) # adding a constant

In [ ]:
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 

In [ ]:
print_model = model.summary()
print(print_model)

In [72]:
df.columns

Index(['Unnamed: 0', 'V1', 'athlete', 'gender', 'run', 'location', 'date',
       'TurnNr', 'MinimalRadius', 'VelocityAtMinRadius', 'VelocityAtTurnEntry',
       'VelocityAtTurnExit', 'TimeStarttoEnd', 'TimeStartToMinRad',
       'TimeMinRadToEnd', 'GlideTime_endtostart', 'TimeStarttoEnd_2',
       'Initiation', 'Completion', 'Gliding', 'Unnamed0', 'vectortonext',
       'vector2Dtonext', 'GATEDISTANCE', 'dist2Dtonext', 'vectortonextnorm',
       'vector2Dtonextnorm', 'STEEPNESS', 'azitonext', 'slopechangetonext',
       'angletonext', 'projtonext', 'projpttonext', 'offsettonext',
       'vectortonextnext', 'vector2Dtonextnext', 'dist3Dtonextnext',
       'dts2Dtonextnext', 'vectortonextnextnorm', 'vector2Dtonextnextnorm',
       'slopetonextnext', 'azitonextnext', 'HORIZONTALGATEDISTANCE',
       'VERTICALGATEDISTANCE', 'regularbs1orfs2', 'steepness_A', 'filter_$',
       'goofy2_regular1', '@2Ferse_3Zehe_4Ferse', 'Zehen1_Fersen2',
       'DistanzSchätz'],
      dtype='object')